In [1]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow_docs.vis import embed

from tensorflow import keras
from keras import layers

from abc import ABC, abstractmethod

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import sys
import os

In [3]:
TEST_RATIO = 0.3
VALIDATION_RATIO = 0.25
TRAIN_RATIO = 1 - TEST_RATIO - VALIDATION_RATIO

HOLDOUT_RATIO = 0.04

MAX_SEQ_LENGTH = 150
IMG_SIZE = 128

BATCH_SIZE = 128
CLASSES = 4

EPOCHS = 40#40

#FEATURE_DIR = f""
#FEATURE_DIR = f"../Features/InceptionV3_SEQUENCE/{MAX_SEQ_LENGTH}"
MODEL_DIR = f"../Models/List"


SYS_OUT = sys.stdout

CNNs

In [4]:
def build_feature_extractor_DenseNet():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

In [5]:
FEATURE_EXTRACTOR = build_feature_extractor_DenseNet()

DENSE

In [6]:
def get_sequence_model_DENSE(num_feature_, layers):
    layer_size = np.size(layers)
    
    dense_model = keras.Sequential()

    dense_model.add(keras.layers.Dense(layers[0], activation='relu', name='dense_0', input_dim=num_feature_))

    for i in range(1, layer_size):
        if (layers[i] >= 64):
            dense_model.add(keras.layers.Dropout(0.3))
        dense_model.add(keras.layers.Dense(layers[i], name=f'dense_{i}',activation='relu'))

    dense_model.add(keras.layers.Dense(CLASSES, activation="softmax", name=f'dense_{layer_size}'))

    dense_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return dense_model

def run_experiment_DENSE(DIR, num_feature_, layers):
    checkpoint = keras.callbacks.ModelCheckpoint(
        DIR, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model_DENSE(num_feature_, layers)

    seq_model.load_weights(DIR)

    return seq_model

In [7]:
MODEL = run_experiment_DENSE(f'../Models/Buenos/2672_DENSE_DenseNet121_[512]_SEQ0_Feature_num1024/video_classifier', 1024, np.array([512]))

In [8]:
import cv2

center_crop_layer = layers.CenterCrop(IMG_SIZE, IMG_SIZE)

def MostrarImagen(frame):
    cv2.imshow('Frame', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = cropped.numpy().squeeze()
    return cropped

def load_image(path):
    image = cv2.imread(path)
    frame = crop_center(image)
    frame = frame[:, :, [2, 1, 0]]
    return np.array(frame)

In [18]:
frame = load_image(f'animation.jpg')
#MostrarImagen(frame)

frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)

In [23]:
probabilities = MODEL.predict(frame_features)[0]
print(probabilities)

1/1 [==============================] - 0s 18ms/step
[1.9849311e-05 2.1447677e-05 9.0628513e-04 9.9905235e-01]


In [9]:
for i in range(4):
    frame = load_image(f'{i}.jpg')
    frame_features = FEATURE_EXTRACTOR.predict(frame[None, :], verbose=0)
    probabilities = MODEL.predict(frame_features)[0]
    for num in probabilities:
        print(f'{num:.3f}|', end='')
    print('')


1/1 [==============================] - 0s 39ms/step
0.000|0.000|0.000|1.000|
1/1 [==============================] - 0s 15ms/step
0.000|0.003|0.000|0.996|
1/1 [==============================] - 0s 16ms/step
0.000|0.000|0.000|1.000|
1/1 [==============================] - 0s 16ms/step
0.000|0.019|0.000|0.981|
